In [3]:
!pip install scikit-learn chardet pyproj folium

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import folium
from folium.plugins import MarkerCluster
from pyproj import Proj

# Cargar los datos
data = pd.read_csv('datos_madrid.csv', encoding='utf-8', encoding_errors='replace')

# Preprocesar datos
data = data.dropna()  # Eliminar filas con valores nulos
data['fecha'] = pd.to_datetime(data['fecha'], format='%d/%m/%Y')  # Convertir fechas

# Eliminar columnas de tiempo o datos no numéricos que no sean catególicos
if 'hora' in data.columns:
    data = data.drop(columns=['hora'])

# Transformar variables categóricas a numéricas
categorical_columns = ['distrito', 'tipo_accidente', 'tipo_vehiculo', 'tipo_persona', 'rango_edad', 'sexo', 'lesividad']
data = pd.get_dummies(data, columns=categorical_columns)

# Definir la proyección UTM para la zona específica de Madrid
proj_utm = Proj(proj='utm', zone=30, ellps='WGS84')  # Zona 30T es típica para Madrid

# Convertir coordenadas UTM a geográficas (latitud, longitud)
lon, lat = proj_utm(data['coordenada_x_utm'].values, data['coordenada_y_utm'].values, inverse=True)
data['latitud'] = lat
data['longitud'] = lon

# Verificar y limpiar valores no válidos
data = data.replace([np.inf, -np.inf], np.nan).dropna(subset=['latitud', 'longitud'])

# Selección de variables (solo latitud y longitud)
X = data[['latitud', 'longitud']]
y = np.zeros(len(data))  # Inicializamos con ceros para luego actualizar con predicciones

# Dividir datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear y entrenar el modelo
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
print(f'Precisión del modelo: {accuracy_score(y_test, y_pred)}')

# Agregar las predicciones al DataFrame original
data['probabilidad_accidente'] = model.predict(X)

# Crear un mapa de Folium centrado en Madrid
madrid_map = folium.Map(location=[40.4168, -3.7038], zoom_start=12)

# Crear un grupo de marcadores con MarkerCluster
marker_cluster = MarkerCluster().add_to(madrid_map)

# Agregar marcadores para cada accidente al grupo MarkerCluster
for index, row in data.iterrows():
    folium.Marker(
        location=[row['latitud'], row['longitud']],
        popup=f"Probabilidad de accidente: {row['probabilidad_accidente']}"
    ).add_to(marker_cluster)

# Guardar el mapa como un archivo HTML
madrid_map.save('mapa_accidentes_madrid.html')


Precisión del modelo: 1.0
